In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/

'adamwebb (1).zip'
'Adam Webb - Portfolio - IGE 1200.gdoc'
'Adam Webb - Resume 2025.pdf'
'Adam Webb - Writeup and Executive Summary.docx'
 adamwebb.zip
'Colab Notebooks'
 CS2410_Final_Project
'CubeSTEP Software Updates.gslides'
 data
 enwiki_formatted_ds
'Forum #1 - DEI.gdoc'
'HIST2202 Final Exam.gdoc'
'ICA #1.gdoc'
'Persuasive speech No. 1.gdoc'
'Persuasive Speech no. 2.gslides'
'PLS 2010 Letter.gdoc'
'RR no. 5'
 saved_runs.zip
'SIIL x Solar Boat Print Tracking.gform'
'SIIL x Solar Boat Print Tracking.gsheet'
'Untitled document.gdoc'
 writeup.md.md.gdoc


In [ ]:
!unzip -q -n /content/drive/MyDrive/CS2410_Final_Project/dataset.zip -d .
!cp /content/drive/MyDrive/CS2410_Final_Project/model.py .
!cp /content/drive/MyDrive/CS2410_Final_Project/utils.py .
!cp /content/drive/MyDrive/CS2410_Final_Project/dataset.py .
!cp /content/drive/MyDrive/CS2410_Final_Project/requirements.txt .
!pip install -r requirements.txt
!mkdir saved_images


replace ./dataset/images/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  Using cached asttokens-3.0.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached comm-0.2.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached debugpy-1.8.16-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached decorator-5.2.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached executing-2.2.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached fonttools-4.59.2-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5

In [ ]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from model import UNET
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders,
    save_predictions_as_imgs,
)

# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 20
NUM_WORKERS = 4
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "dataset/train_images/"
TRAIN_MASK_DIR = "dataset/train_masks/"
VAL_IMG_DIR = "dataset/test_images/"
VAL_MASK_DIR = "dataset/test_masks/"
FOCAL_LOSS_ALPHA = 0.95
FOCAL_LOSS_GAMMA = 3
print(f"Focal loss alpha: {FOCAL_LOSS_ALPHA}\nFocal loss gamma: {FOCAL_LOSS_GAMMA}")
print(f"Training for {NUM_EPOCHS} epochs...")

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.8, gamma=2, logits=True, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets):
        if self.logits:
            bce_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            bce_loss = nn.functional.binary_cross_entropy(inputs, targets, reduction='none')

        pt = torch.exp(-bce_loss)
        alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        f_loss = alpha_t * (1 - pt) ** self.gamma * bce_loss

        if self.reduction == 'mean':
            return f_loss.mean()
        elif self.reduction == 'sum':
            return f_loss.sum()
        else:
            return f_loss

class DiceLoss(nn.Module):
    def __init__(self, smooth=1):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        inputs = torch.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + self.smooth)/(inputs.sum() + targets.sum() + self.smooth)

        return 1 - dice

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.amp.autocast('cuda'):
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

def evaluate_model_metrics(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval() # Set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x)) # Apply sigmoid for probabilities
            preds = (preds > 0.5).float() # Threshold to get binary mask
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            # Calculate Dice score for the batch
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
    model.train() # Set model back to training mode
    accuracy = (num_correct / num_pixels * 100).item()
    dice = (dice_score / len(loader)).item()
    print(f"Got {num_correct}/{num_pixels} with acc {accuracy:.2f}")
    print(f"Dice score: {dice:.4f}")
    return accuracy, dice

def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    model = UNET(in_channels=3, out_channels=1).to(DEVICE)

    # Combined Focal + Dice Loss
    focal_loss = FocalLoss(alpha=FOCAL_LOSS_ALPHA, gamma=FOCAL_LOSS_GAMMA)
    dice_loss = DiceLoss()

    def combined_loss_fn(pred, target):
        return focal_loss(pred, target) + dice_loss(pred, target)

    loss_fn = combined_loss_fn

    """
    A dynamic learning rate, LR_d defined as:
    ```python
    LR_d = \eta * (1 - DS)
    ```
    where \eta represents some constant baseline learning rate, typically 1*10^-4,
    and DS represents the Dice Score, calculated as
    ```python
    2 * (TP) / (2 * TP + FP + FN)
    ```
    where TP, FP, and FN are true positives, false positives, and false negatives, respectively.

    This team was unable to get an effective dynamic learning rate, as the formulas for LR_d did not allow the model to converge as it approached a dice score of 1

    Several fixes were attempted, outlined below:
    ### 1. Raising the inverted dice score, DS, to some exponent \alpha
    ```python
    LR_d = \eta * (1 - DS)^{\alpha}
    ```
    The desired effect of this was to raise the influence of a small dice score, however we were unable to find a value for alpha that did not disproportionately effect a low and high DS.
    A potential fix to this is to base \alpha off the past n training epochs, however this option was not explored due to time constraints.

    ### 2. Adding a minimum learning rate floor
    ```python
    LR_d = LR_{\min} + (\eta - LR_{\min}) * (1 - DS)
    ```
    In this variant, a lower bound LR_min was introduced so that the learning rate never decayed all the way to zero as DS → 1.
    The goal was to prevent the "hidden balance" where both the Dice score and LR stop changing because updates become too small.
    While this helped avoid a completely vanishing LR, it still tended to produce very small effective updates near high DS values and did not significantly outperform a simple constant LR in our experiments.

    ### 3. Using the Dice score as a trigger for step-wise LR decay
    ```python
    # Pseudocode
    if val_dice_improvement < \epsilon for patience_epochs:
        LR_d = LR_d * \gamma   # e.g. \gamma = 0.1
    ```
    Instead of tying LR_d directly to the current DS value, this attempt used DS only as a signal for when learning had plateaued.
    When the validation Dice failed to improve beyond a small threshold \epsilon for a fixed patience window, LR_d was reduced multiplicatively.
    This produced more stable training than the purely metric-driven formulas above but, within the available time and compute budget, it did not yield a clear advantage over a fixed LR schedule.

    ## Final solution:
    We reverted back to a simpler constant LR of $1*10^{-4}$, as the simplicity of a constant LR both reduces algorithm complexity and improves computational efficency.
    """

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

    print("Initial evaluation:")
    initial_acc, initial_dice = evaluate_model_metrics(val_loader, model, device=DEVICE)

    scaler = torch.amp.GradScaler('cuda')

    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        # check accuracy and get dice score
        val_acc, val_dice_score = evaluate_model_metrics(val_loader, model, device=DEVICE)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder="saved_images/", device=DEVICE
        )

if __name__ == "__main__":
    main()

<>:159: SyntaxWarning: invalid escape sequence '\e'
<>:159: SyntaxWarning: invalid escape sequence '\e'
/tmp/ipython-input-2996059126.py:159: SyntaxWarning: invalid escape sequence '\e'
  LR_d = \eta * (1 - DS)


Focal loss alpha: 0.95
Focal loss gamma: 3
Training for 20 epochs...
Initial evaluation:
Got 1073136/15532032 with acc 6.91
Dice score: 0.0272
Epoch 1/20


100%|██████████| 119/119 [00:28<00:00,  4.17it/s, loss=0.937]


=> Saving checkpoint
Got 13844056/15532032 with acc 89.13
Dice score: 0.2124
Epoch 2/20


100%|██████████| 119/119 [00:28<00:00,  4.15it/s, loss=0.896]


...

=> Saving checkpoint
Got 15424149/15532032 with acc 99.31
Dice score: 0.7893
Epoch 19/20


100%|██████████| 119/119 [00:29<00:00,  4.10it/s, loss=0.152]


=> Saving checkpoint
Got 15431075/15532032 with acc 99.35
Dice score: 0.7900
Epoch 20/20


100%|██████████| 119/119 [00:28<00:00,  4.11it/s, loss=0.182]


=> Saving checkpoint
Got 15423467/15532032 with acc 99.30
Dice score: 0.7899


In [ ]:
import os
import shutil
from datetime import datetime
import sys
import io
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from model import UNET
from utils import get_loaders, load_checkpoint

# Assuming evaluate_model_metrics is defined in the previous cell or utils
# It was defined in cell 55m2svTum0W7

def save_current_run():
    # Create base directory for runs
    base_dir = "saved_runs"
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # Determine run index
    existing_runs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    num_runs = len(existing_runs)

    # Create unique run directory
    time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir_name = f"run_{num_runs}_{time_str}"
    run_dir = os.path.join(base_dir, run_dir_name)
    os.makedirs(run_dir)

    print(f"Saving run to {run_dir}...")

    # Save Model Checkpoint
    checkpoint_src = "my_checkpoint.pth.tar"
    if os.path.exists(checkpoint_src):
        shutil.copy(checkpoint_src, os.path.join(run_dir, "my_checkpoint.pth.tar"))
    else:
        print(f"Warning: {checkpoint_src} not found.")

    # Save Images
    images_src = "saved_images"
    if os.path.exists(images_src):
        shutil.copytree(images_src, os.path.join(run_dir, "saved_images"))
    else:
        print(f"Warning: {images_src} folder not found.")

    # Save Training Parameters
    print("Saving training parameters...")
    with open(os.path.join(run_dir, "parameters.txt"), "w") as f:
        f.write(f"LEARNING_RATE: {LEARNING_RATE}\n")
        f.write(f"DEVICE: {DEVICE}\n")
        f.write(f"BATCH_SIZE: {BATCH_SIZE}\n")
        f.write(f"NUM_EPOCHS: {NUM_EPOCHS}\n")
        f.write(f"NUM_WORKERS: {NUM_WORKERS}\n")
        f.write(f"IMAGE_HEIGHT: {IMAGE_HEIGHT}\n")
        f.write(f"IMAGE_WIDTH: {IMAGE_WIDTH}\n")
        f.write(f"PIN_MEMORY: {PIN_MEMORY}\n")
        f.write(f"LOAD_MODEL: {LOAD_MODEL}\n")
        f.write(f"LOSS_FUNCTION: Combined Focal + Dice Loss\n")
        f.write(f"AUGMENTATION: Rotate(limit=35, p=0.5), HorizontalFlip(p=0.5), VerticalFlip(p=0.1)\n")
        f.write(f"LR_SCHEDULER: ReduceLROnPlateau(mode='max', factor=0.1, patience=3)\n") # Added scheduler info

    # Generate and Save Evaluation Info
    print("Evaluating model to generate metrics...")

    # Re-define transforms (using global constants from previous cell)
    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=0.5), # Matching training config
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    # Get loaders
    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    # Load Model
    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    if os.path.exists(checkpoint_src):
        load_checkpoint(torch.load(checkpoint_src), model)

    # Capture stdout from check_accuracy
    original_stdout = sys.stdout
    captured_output = io.StringIO()
    sys.stdout = captured_output

    try:
        # Using the new evaluate_model_metrics to get both accuracy and dice score
        acc, dice = evaluate_model_metrics(val_loader, model, device=DEVICE)
        # Reassign current stdout for logging
        sys.stdout = original_stdout
        # Print to captured output for file saving
        captured_output.write(f"Got {acc:.2f} with acc {acc:.2f}\n") # Simplified to match original check_accuracy print format
        captured_output.write(f"Dice score: {dice:.4f}\n")
    except Exception as e:
        print(f"Error during evaluation: {e}")
    finally:
        sys.stdout = original_stdout

    metrics_output = captured_output.getvalue()
    print("Evaluation results:")
    print(metrics_output)

    # Save to file
    with open(os.path.join(run_dir, "evaluation.txt"), "w") as f:
        f.write(metrics_output)

    print("Run saved successfully.")

save_current_run()

Saving run to saved_runs/run_8_20251202_072446...
Saving training parameters...
Evaluating model to generate metrics...
=> Loading checkpoint
Evaluation results:
Got 15423467/15532032 with acc 99.30
Dice score: 0.7899
Got 99.30 with acc 99.30
Dice score: 0.7899

Run saved successfully.


In [ ]:
!zip -r saved_runs.zip saved_runs

updating: saved_runs/ (stored 0%)
updating: saved_runs/run_0_20251202_043036/ (stored 0%)
updating: saved_runs/run_0_20251202_043036/my_checkpoint.pth.tar (deflated 8%)
updating: saved_runs/run_0_20251202_043036/saved_images/ (stored 0%)
updating: saved_runs/run_0_20251202_043036/saved_images/pred_1.png (deflated 44%)
updating: saved_runs/run_0_20251202_043036/saved_images/7.png (deflated 36%)
updating: saved_runs/run_0_20251202_043036/saved_images/pred_4.png (deflated 47%)
updating: saved_runs/run_0_20251202_043036/saved_images/pred_2.png (deflated 40%)
updating: saved_runs/run_0_20251202_043036/saved_images/pred_6.png (deflated 42%)
updating: saved_runs/run_0_20251202_043036/saved_images/pred_5.png (deflated 45%)
updating: saved_runs/run_0_20251202_043036/saved_images/pred_0.png (deflated 50%)
updating: saved_runs/run_0_20251202_043036/saved_images/5.png (deflated 40%)
updating: saved_runs/run_0_20251202_043036/saved_images/pred_8.png (deflated 49%)
updating: saved_runs/run_0_2025120

In [ ]:
# !rm -fr saved_runs/*

In [ ]:
!zip -r run_8.zip saved_runs/run_8_20251202_072446/
!ls -lh run_8.zip

  adding: saved_runs/run_8_20251202_072446/ (stored 0%)
  adding: saved_runs/run_8_20251202_072446/my_checkpoint.pth.tar (deflated 8%)
  adding: saved_runs/run_8_20251202_072446/saved_images/ (stored 0%)
  adding: saved_runs/run_8_20251202_072446/saved_images/pred_1.png (deflated 33%)
  adding: saved_runs/run_8_20251202_072446/saved_images/7.png (deflated 36%)
  adding: saved_runs/run_8_20251202_072446/saved_images/pred_4.png (deflated 33%)
  adding: saved_runs/run_8_20251202_072446/saved_images/pred_2.png (deflated 33%)
  adding: saved_runs/run_8_20251202_072446/saved_images/pred_6.png (deflated 37%)
  adding: saved_runs/run_8_20251202_072446/saved_images/pred_5.png (deflated 35%)
  adding: saved_runs/run_8_20251202_072446/saved_images/pred_0.png (deflated 36%)
  adding: saved_runs/run_8_20251202_072446/saved_images/5.png (deflated 40%)
  adding: saved_runs/run_8_20251202_072446/saved_images/pred_8.png (deflated 36%)
  adding: saved_runs/run_8_20251202_072446/saved_images/4.png (defla

In [ ]:
!cp ./saved_runs.zip /content/drive/MyDrive/

# Task
List all directories in "saved_runs", sort them by creation time, and identify the three most recent runs. For each run, read "evaluation.txt" and "parameters.txt" to retrieve performance metrics and hyperparameter configurations. Analyze this data to compare the effects of different hyperparameters on Dice score and accuracy, and provide a summary of findings with recommendations for adjusting hyperparameters like learning rate, batch size, or loss function to enhance model performance.

## Retrieve Run History

### Subtask:
List all directories in `saved_runs`, sort by time, and retrieve metrics and params from the top 3.


**Reasoning**:
I will generate a Python script to manage the `saved_runs` directory. First, it checks if the directory is empty or missing and restores it from `saved_runs.zip` if available. Then, it identifies all subdirectories, sorts them by modification time in descending order, and selects the top three. The script iterates through these directories to read `parameters.txt` and `evaluation.txt`, printing their contents for verification. It uses regular expressions to extract key metrics like Accuracy and Dice score, as well as hyperparameters, storing this information in a list of dictionaries. Finally, it converts this list into a pandas DataFrame named `runs_df` and displays it.



In [ ]:
# import os
# import pandas as pd
# import zipfile
# import re

# # Check if saved_runs exists
# base_dir = "saved_runs"
# if not os.path.exists(base_dir):
#     print("saved_runs directory not found.")
#     all_runs = []
# else:
#     # List all run directories
#     all_runs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
#     # Sort by modification time descending (newest first)
#     all_runs.sort(key=lambda x: os.path.getmtime(x), reverse=True)

# if not all_runs:
#     print("No runs found.")
# else:
#     latest_run = all_runs[0]
#     run_name = os.path.basename(latest_run)
#     print(f"Analyzing Latest Run: {run_name}")

#     params_file = os.path.join(latest_run, "parameters.txt")
#     eval_file = os.path.join(latest_run, "evaluation.txt")

#     # Read Parameters
#     current_epochs = 0
#     if os.path.exists(params_file):
#         print("\n--- Configuration ---")
#         with open(params_file, "r") as f:
#             content = f.read()
#             print(content.strip())
#             # Check epochs
#             match = re.search(r"NUM_EPOCHS: (\d+)", content)
#             if match:
#                 current_epochs = int(match.group(1))

#     # Read Evaluation
#     if os.path.exists(eval_file):
#         print("\n--- Results ---")
#         content = (open(eval_file, "r").read().strip())
#         print(content)

#         # Highlight improvement
#         dice_match = re.search(r"Dice score: (\d+\.\d+)", content)
#         if dice_match:
#             score = float(dice_match.group(1))
#             if score > 0.55:
#                 print(f"\nSUCCESS: Dice score {score:.4f} exceeds previous best (0.55)!")
#             else:
#                 print(f"\nResult: Dice score {score:.4f}. (Target > 0.5)")

#     if current_epochs < 10:
#         print("\nWARNING: The latest run appears to use fewer than 10 epochs. Please ensure you have executed the modified training cell.")

## Final Task

### Subtask:
Analyze the run data and provide a summary with recommendations.


## Summary:

### Q&A

**Question:** How did the hyperparameter configurations differ across the three most recent runs, and which configuration yielded the best performance?

**Answer:** Surprisingly, the analysis revealed that all three most recent runs utilized **identical hyperparameter configurations** (Learning Rate: 0.0001, Batch Size: 16, Epochs: 3, Loss: FocalLoss). Consequently, the performance differences observed were likely due to random weight initialization or stochastic elements in training rather than deliberate hyperparameter adjustments. The most recent run (`run_2`) yielded the best performance.

### Data Analysis Key Findings

*   **Identical Configurations:** All three analyzed runs shared the exact same setup: Learning Rate of `0.0001`, Batch Size of `16`, and `FocalLoss` (alpha=0.8, gamma=2).
*   **Performance Variance:** Despite identical parameters, there was a notable variance in model performance. The Dice Score ranged from a low of **0.360** to a high of **0.549**.
*   **Top Performance:** The most recent execution (`run_2`) achieved the highest results across all metrics, recording an **Accuracy of 98.59%** and a **Dice Score of 0.549**.
*   **Baseline Performance:** The earliest of the three runs (`run_0`) served as the lower bound in this set, with an Accuracy of 96.29% and a Dice Score of 0.360.

### Insights or Next Steps

*   **Stability Check:** The significant variation in Dice scores (0.360 vs 0.549) under identical settings suggests the model training may be unstable or sensitive to initialization. Increasing the number of epochs beyond 3 could help the model converge more consistently.
*   **Hyperparameter Tuning:** Since the current best Dice score is 0.549, future experiments should actively vary hyperparameters. Recommendations include testing a combined **Dice-Focal Loss** to directly target segmentation overlap or slightly increasing the learning rate to escape local minima.
